In [ ]:
import pandas as pd
import numpy as np
import math
import json
import matplotlib.pyplot as plt
%matplotlib inline
from clean_data_2 import *

import plotly.express as px

#### Frame the Problem


This data set contains simulated data that mimics customer behavior on the Starbucks rewards mobile app. Once every few days, Starbucks sends out an offer to users of the mobile app. An offer can be merely an advertisement for a drink or an actual offer such as a discount or BOGO (buy one get one free). Some users might not receive any offer during certain weeks.

Not all users receive the same offer, and that is the challenge to solve with this data set.

Your task is to combine transaction, demographic and offer data to determine which demographic groups respond best to which offer type. This data set is a simplified version of the real Starbucks app because the underlying simulator only has one product whereas Starbucks actually sells dozens of products.

Every offer has a validity period before the offer expires. As an example, a BOGO offer might be valid for only 5 days. You'll see in the data set that informational offers have a validity period even though these ads are merely providing information about a product; for example, if an informational offer has 7 days of validity, you can assume the customer is feeling the influence of the offer for 7 days after receiving the advertisement.

You'll be given transactional data showing user purchases made on the app including the timestamp of purchase and the amount of money spent on a purchase. This transactional data also has a record for each offer that a user receives as well as a record for when a user actually views the offer. There are also records for when a user completes an offer.

Keep in mind as well that someone using the app might make a purchase through the app without having received an offer or seen an offer.

Example
To give an example, a user could receive a discount offer buy 10 dollars get 2 off on Monday. The offer is valid for 10 days from receipt. If the customer accumulates at least 10 dollars in purchases during the validity period, the customer completes the offer.

However, there are a few things to watch out for in this data set. Customers do not opt into the offers that they receive; in other words, a user can receive an offer, never actually view the offer, and still complete the offer. For example, a user might receive the "buy 10 dollars get 2 dollars off offer", but the user never opens the offer during the 10 day validity period. The customer spends 15 dollars during those ten days. There will be an offer completion record in the data set; however, the customer was not influenced by the offer because the customer never viewed the offer.

Cleaning
This makes data cleaning especially important and tricky.

You'll also want to take into account that some demographic groups will make purchases even if they don't receive an offer. From a business perspective, if a customer is going to make a 10 dollar purchase without an offer anyway, you wouldn't want to send a buy 10 dollars get 2 dollars off offer. You'll want to try to assess what a certain demographic group will buy when not receiving any offers.

Final Advice
Because this is a capstone project, you are free to analyze the data any way you see fit. For example, you could build a machine learning model that predicts how much someone will spend based on demographics and offer type. Or you could build a model that predicts whether or not someone will respond to an offer. Or, you don't need to build a machine learning model at all. You could develop a set of heuristics that determine what offer you should send to each customer (i.e., 75 percent of women customers who were 35 years old responded to offer A vs 40 percent from the same demographic to offer B, so send offer A).

### Desired Result

The desired result will be an app that will filter offers based on the selected demographics by the user. The app is to be used by starbuck's marketing department to select the right offer for the right demographic. 

In this jupyter notebook we will explore the nuances of our data and understand the different aspects of customers, offers, and transactions

In [ ]:
from jupyterthemes import jtplot
jtplot.style(theme='monokai', context='notebook', ticks=True, grid=False)

#### Import data

In [ ]:
portfolio = pd.read_json('data/portfolio.json', orient='records', lines=True)
profile = pd.read_json('data/profile.json', orient='records', lines=True)
transcript = pd.read_json('data/transcript.json', orient='records', lines=True)

I am using the clean_portfolio functions created in the clean_data_2.py script to return a clean portfolio datagrame.
I will do the same for profile and transcript

In [ ]:
portfolio = clean_portfolio(portfolio)

In [ ]:
portfolio

In [ ]:
profile = clean_profile(profile)

In [ ]:
profile.head()

In [ ]:
transcript.head()

In [ ]:
transcript = clean_transcript(transcript)

In [ ]:
transcript 

In [ ]:
df = merge_datasets(portfolio, profile, transcript)

In [ ]:
df

## Analysis of our data set

our dataset has 306,534 rows. These rows combine event and offer data in a single dataset to make it easier for analysis 

In [ ]:
df.shape

### Story of 1 customer 

Below we see the story of 1 customer, the behaviour of a customer for the duration of the test. Time 0 indicates the start of the test and the last row is the last event that happened. We have randomly chosen a customer at the age of 75 that has received multiple offers and we have a historic view of how she responded. Ideally we want to replicate that pythonically for every customer

In [ ]:
df[df.customer_id =='78afa995795e4d85b5d9ceeca43f5fef']

In the below table we see the behaviour of our selected customer for one specific offer. We can see that the offer was succesful

In [ ]:
df[(df.offer_id=='B3') & (df.customer_id=='78afa995795e4d85b5d9ceeca43f5fef')]

#### Which offers get completed more 

To asnwer this I will create a bar chart to see the 

In [ ]:
df[df.event_offer_received == 1].offer_type.value_counts().plot(kind='bar', rot=45, figsize=(6,6), alpha=0.6)
plt.title('Offer type received')

#### What is the distribution of the amount people are spending on a transaction?

I will plot a histogram of the average spend for all customers

In [ ]:
df.amount.hist(bins = 25, range=[0, 50], alpha = 0.7)
plt.title('Average Spend')

Most of customers seem to be spending 0-10 dollars in a starbucks shop which makes sense, however there are customers that spend more than 10 or even 20 in a single transaction. I guess they're having muffins with their lattes 

#### What is the income distribution in our dataset ?

It is important to understand the income distribution in order to get a better understanding of what of our customer base

In [ ]:
df.income.hist()
plt.title('Income Distribution')

It seems that majority of our customers have an income below 80K. The median US income in the US in 2016 was reported as 31000 dollars. That means our customer base has an above average income

#### Is there a correlation between amount spent and income? 

For small transactions there doesn't seem to be a correlation between spend an income. However for purchases above 200 dollars there is a strong positive correlation with income and amount spent

In [ ]:
plt.scatter(x='income', y='amount',data=df)

In [ ]:
df_amount = df[df.amount > 200]

Plotting just the amount spent above 200 dollars the correlation is  obvious

In [ ]:
px.scatter(df_amount,x='income', y='amount',)

### some final points

For the final part I want to use data aggregated per customer to understand some more points. Instead of just seeing the amount someone spends in a single purchase I would like to see the correlation with the total amount someone spends at starbucks.

In [ ]:
customers = per_customer_data(df, profile)

In [ ]:
customers

#### Does income group has a releationship with the total expense of a customer ? 

For that I will generate a graph by income group to see if with increasing income groups the total spend is 

In [ ]:
customers.groupby('income_group')['total_expense'].mean().plot(kind='bar')

From the above plot I see an interesting increase in total expense with increase in income

#### Does age group has an effect on total expense ? 

In [ ]:
customers.groupby('age_group')['total_expense'].mean().plot(kind='bar')

I don't particulary see a clear increase in spend with the increase in income. However our customers seem to have a slight tendency to overall spend more the older they are. I presume the older someone is the higher their income

#### Does gender affect total expense ? 

Girls seem to be spending a little more than boys overall at starbucks. That can be an interesting insight

In [ ]:
customers.groupby('gender')['total_expense'].mean().plot(kind='bar')

#### Initial conclusion from the analysis

Gender, age, and income are all factors that affect the total expense of a customer and they play a role in the kind of offer that are more succesful. 
Our ranking based approach will take into account those demographics when deciding the output of the offers it recommends.